# Wynn Resort Forecast Model Tutorial
## Part 2: Building the RNN Forecast Model

This notebook demonstrates the forecasting engine that will power our AI system.

In [ ]:
# Install required packages
!pip install numpy pandas matplotlib

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import json

## Step 1: Generate Historical Data
Creating realistic Vegas hotel patterns with hourly granularity

In [ ]:
def generate_historical_data():
    """Generate 6 months of hourly Vegas hotel data"""
    
    # Generate hourly data from 6 months ago to now
    start_date = datetime.now() - timedelta(days=180)
    end_date = datetime.now()
    
    dates = []
    current_date = start_date
    while current_date <= end_date:
        dates.append(current_date)
        current_date += timedelta(hours=1)
    
    data = []
    for date in dates:
        hour = date.hour
        day_of_week = date.weekday()
        
        # Base patterns
        # Room occupancy: Higher on weekends, peaks at night
        if day_of_week >= 4:  # Friday-Sunday
            room_base = 85 + 10 * np.sin(2 * np.pi * hour / 24)
        else:
            room_base = 70 + 15 * np.sin(2 * np.pi * hour / 24)
        
        # Cleaning staff: Peaks during checkout (10-14) and turndown (18-20)
        if 10 <= hour <= 14:
            cleaning_base = 80 + (room_base * 0.8)
        elif 18 <= hour <= 20:
            cleaning_base = 40 + (room_base * 0.3)
        else:
            cleaning_base = 20 + (room_base * 0.1)
        
        # Security: Higher at night, especially weekends
        if 22 <= hour or hour <= 4:
            security_base = 40 + (20 if day_of_week >= 4 else 10)
        else:
            security_base = 20 + (10 if day_of_week >= 4 else 5)
        
        # Add noise
        rooms = max(0, min(100, room_base + np.random.normal(0, 5)))
        cleaning = max(0, cleaning_base + np.random.normal(0, 10))
        security = max(0, security_base + np.random.normal(0, 5))
        
        data.append({
            'datetime': date,
            'rooms': rooms,
            'cleaning': cleaning,
            'security': security
        })
    
    return pd.DataFrame(data)

# Generate the data
historical_data = generate_historical_data()
print(f"Generated {len(historical_data)} hours of historical data")
print(f"Date range: {historical_data['datetime'].min()} to {historical_data['datetime'].max()}")
historical_data.tail()

## Step 2: Build the Forecast Model

In [ ]:
class ForecastRNN:
    def __init__(self):
        self.metrics = ['rooms', 'cleaning', 'security']
        self.model_trained = False
        self.base_patterns = {}
    
    def train(self, historical_data):
        """Train model by analyzing historical patterns"""
        for metric in self.metrics:
            # Calculate hourly averages
            hourly_avg = historical_data.groupby(historical_data['datetime'].dt.hour)[metric].mean()
            
            # Calculate day of week patterns
            dow_avg = historical_data.groupby(historical_data['datetime'].dt.dayofweek)[metric].mean()
            
            # Store patterns
            self.base_patterns[metric] = {
                'hourly': hourly_avg.to_dict(),
                'dow': dow_avg.to_dict(),
                'overall_mean': historical_data[metric].mean(),
                'overall_std': historical_data[metric].std()
            }
        
        self.model_trained = True
        print("Model training complete!")
    
    def predict(self, historical_data, hours=168):
        """Generate hourly forecast for next 168 hours (7 days)"""
        if not self.model_trained:
            self.train(historical_data)
        
        forecast_data = []
        start_time = datetime.now()
        
        for i in range(hours):
            forecast_time = start_time + timedelta(hours=i+1)
            hour = forecast_time.hour
            dow = forecast_time.weekday()
            
            row = {'datetime': forecast_time}
            
            for metric in self.metrics:
                # Combine hourly and day-of-week patterns
                hourly_factor = self.base_patterns[metric]['hourly'][hour]
                dow_factor = self.base_patterns[metric]['dow'][dow]
                base_value = (hourly_factor + dow_factor) / 2
                
                # Add some random variation
                noise = np.random.normal(0, self.base_patterns[metric]['overall_std'] * 0.1)
                value = base_value + noise
                
                # Ensure reasonable bounds
                if metric == 'rooms':
                    value = max(50, min(100, value))
                else:
                    value = max(0, value)
                
                row[metric] = value
            
            forecast_data.append(row)
        
        return pd.DataFrame(forecast_data)

# Create and train the model
model = ForecastRNN()
model.train(historical_data)

## Step 3: Generate Forecast

In [ ]:
# Generate 168-hour forecast
forecast = model.predict(historical_data)
print(f"Generated {len(forecast)} hours of forecast")
forecast.head()

## Step 4: Visualize the Forecast

In [ ]:
# Plot the forecast
fig, axes = plt.subplots(3, 1, figsize=(15, 10))

# Room Occupancy
axes[0].plot(forecast['datetime'], forecast['rooms'], color='goldenrod', linewidth=2)
axes[0].set_title('Room Occupancy Forecast (%)', fontsize=14)
axes[0].set_ylabel('Occupancy %')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(0, 100)

# Cleaning Staff
axes[1].plot(forecast['datetime'], forecast['cleaning'], color='royalblue', linewidth=2)
axes[1].set_title('Cleaning Staff Needed', fontsize=14)
axes[1].set_ylabel('Staff Count')
axes[1].grid(True, alpha=0.3)

# Security Staff
axes[2].plot(forecast['datetime'], forecast['security'], color='crimson', linewidth=2)
axes[2].set_title('Security Staff Needed', fontsize=14)
axes[2].set_ylabel('Staff Count')
axes[2].set_xlabel('Date/Time')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 5: Export Forecast for Frontend

In [ ]:
# Convert to format matching our frontend
def prepare_for_frontend(historical_df, forecast_df):
    """Prepare data in the format expected by frontend"""
    
    # Get last 168 hours of historical data
    historical_recent = historical_df.tail(168).copy()
    
    # Convert datetime to string
    historical_recent['date'] = historical_recent['datetime'].dt.strftime('%Y-%m-%d %H:%M')
    forecast_df['date'] = forecast_df['datetime'].dt.strftime('%Y-%m-%d %H:%M')
    
    # Select only needed columns
    historical_export = historical_recent[['date', 'rooms', 'cleaning', 'security']].to_dict('records')
    forecast_export = forecast_df[['date', 'rooms', 'cleaning', 'security']].to_dict('records')
    
    return {
        'historical': historical_export,
        'forecast': forecast_export
    }

# Prepare the data
frontend_data = prepare_for_frontend(historical_data, forecast)

# Save to JSON file
with open('wynn_forecast_data.json', 'w') as f:
    json.dump(frontend_data, f, indent=2)

print(f"Exported {len(frontend_data['historical'])} historical records")
print(f"Exported {len(frontend_data['forecast'])} forecast records")
print("\nData saved to wynn_forecast_data.json")

## Summary

We've built a forecasting model that:
1. Generates realistic Vegas hotel patterns
2. Trains on historical hourly data
3. Produces 168-hour forecasts
4. Exports data for our frontend

Next steps: Connect this to our WebSocket backend!